In [2]:
import json
import glob
import os
import datetime
import re
import numpy as np
import pandas as pd
from pathlib import Path
from slack_data_loader2 import SlackLoader2, normalize_links
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%load_ext autoreload
%autoreload 2

### data
* "channel"
* "start_message_text"
* "thread_text" (__eou__ __eot__)
* "message_type" = ["top", "first_thread", "thread"]
* "attachment" = 1|0
* "message_number" = (1 - top,)
* "message_text" 
### _target_
emoji, count

In [15]:
BASE_DIR = Path("/home/algis/repos/personal/MOOC/ODS_dump/input/export_Feb_8_2018")


loader = SlackLoader2(BASE_DIR, exclude_channels=[], only_channels=['_jobs'])
print(len(loader.messages))
print(len(loader.threads))
df = pd.DataFrame(loader.threads)

100%|██████████| 18408/18408 [00:02<00:00, 6572.57it/s] | 3501/18408 [00:00<00:00, 34986.76it/s]

18408
5805


In [16]:
df


,end_ts,key,msg_counter,reactions_,start_ts,text
0,NaT,"(C04DA5FUF, 1442327532.000097)",1,{'smiley': 2},2015-09-15 17:32:12.000097,oh shi~ __eou__
1,NaT,"(C04DA5FUF, 1442506275.000177)",1,{'moneybag': 3},2015-09-17 19:11:15.000177,сколько поднял? __eou__
2,NaT,"(C04DA5FUF, 1444204890.000044)",1,{'white_check_mark': 3},2015-10-07 11:01:30.000044,"справа босс той компании которая, как я понима..."
3,NaT,"(C04DA5FUF, 1444921027.000106)",1,{'joy': 3},2015-10-15 17:57:07.000106,звучит так словно их в этих банках вот так вот...
4,NaT,"(C04DA5FUF, 1446024340.000192)",1,{'+1': 1},2015-10-28 12:25:40.000192,"уже не стартап, в Берлине 150 человек работает..."
5,NaT,"(C04DA5FUF, 1447939588.000018)",1,"{'smiley': 1, '+1': 2}",2015-11-19 16:26:28.000018,зато hr симпатичная :simple_smile: __eou__
6,NaT,"(C04DA5FUF, 1447939629.00002)",1,{'joy': 4},2015-11-19 16:27:09.000020,<@U041SH27M>: с такими вопросами тебя не возьм...
7,NaT,"(C04DA5FUF, 1448143014.000121)",1,{'feelsgood': 1},2015-11-22 00:56:54.000121,"я думаю насыщение рынка профессий что java, чт..."
8,NaT,"(C04DA5FUF, 1448456390.000172)",1,{'smiley': 1},2015-11-25 15:59:50.000172,Поясню почему сложно с распространением данных...
9,NaT,"(C04DA5FUF, 1449487670.000253)",1,{'+1': 5},2015-12-07 14:27:50.000253,"<!here|@here> итак, сейчас был первый случай б..."


In [14]:
loader.threads[2]

{'key': ('C04DA5FUF', 1444204890.000044),
 'msg_counter': 1,
 'reactions_': {'white_check_mark': 3},
 'text': 'справа босс той компании которая, как я понимаю, постила эту вакансию __eou__'}